# Superpermutaions

Ever since I watched [this video](https://www.youtube.com/watch?v=OZzIvl1tbPo) I've been fascinated with [superpermutations](https://en.wikipedia.org/wiki/Superpermutation). Put simply, superpermutations are numbers which contains every permutation of a list of number from 1 to N. This can be accomplished trivially by just concatenating every permutation in a list, but the interesting question is what is the shortest possible superpermutation for a given number N? For me they are a particualarly exciting problem to think about because a lot of discussion about them is relatively recent and breakthroughs are being made mostly by amatuer mathematicians. It's a complicated problem but one that is very accessible to dive into. 

In [2]:
import termcolor as tc

colorCode = {
    '1': 'yellow',
    '2': 'red',
    '3': 'blue',
    '4': 'green',
    '5': 'magenta',
    '6': 'cyan'
}

def intList_to_string(intList):
    return ''.join([str(x) for x in intList])
    
class SuperPermutation():
    
    def __init__(self, n, text, coding):
        self.n = n
        self.coding = coding
        self.text = text
        self.k = len(text)
    
    def __str__(self):
        return ('SP: N:' + str(self.n) + ' K:' + str(self.k) + '\n' 
        + ''.join([tc.colored(char, self.coding[char], 'on_grey') for char in self.text]))        

In order to better visualize what a superpermutation is I've written a little class that will display a given N, it's superpermution length K, and color code the sequence. We are also going to be printing a lot of list of integers so I've written a shorthand function to do that.

In [3]:
sp_one = SuperPermutation(1, '1', colorCode)
sp_two = SuperPermutation(2, '121', colorCode)
sp_three = SuperPermutation(3, '123121321', colorCode)
sp_four = SuperPermutation(4, '123412314231243121342132413214321', colorCode)

In [4]:
print(sp_one)
print(sp_two)
print(sp_three)
print(sp_four)

SP: N:1 K:1
1
SP: N:2 K:3
121
SP: N:3 K:9
123121321
SP: N:4 K:33
123412314231243121342132413214321


From this we can notice two things, K is growing exponentially with N, and that these superpermutations are palindromes! At least for now...
Rather than manually entering our superpermutations it would be nice to have an algorithm to generate them.

In [5]:
def recursive_solver(input_n):
    
    def is_permutation(sequence):
        seen = {}
        for element in sequence:
            if element in seen:
                return False
            seen[element] = 1
        return True
        
    def combine_overlap(a, b):
        max_overlap = int((len(b) - 1) / 2)
        for idx in range(max_overlap, 0, -1):
            if a[-idx:] == b[:idx]:
                return a + b[idx:]
        
    def recur(n):
        if n == 2:
            return [1, 2, 1]
        else:
            sub_sp = recur(n - 1)
            stage_one_list = []
            for idx in range(0, len(sub_sp) - 1):
                sub_sub_sp = sub_sp[idx: idx + n - 1]
                if is_permutation(sub_sub_sp) and len(sub_sub_sp) == n -1:
                    stage_one_list.append(sub_sub_sp)
            stage_two_list = []
            for sub in stage_one_list:
                stage_two_list.append(sub + [n] + sub)
            super_permutation = stage_two_list[0]
            for idx in range(1, len(stage_two_list)):
                super_permutation = combine_overlap(super_permutation, stage_two_list[idx])
            return super_permutation
        
    return recur(input_n)

The above function is a recursive method to generate superpermutations. It works as follows:

["First, the superpermutation of order n-1 is split into its individual permutations in the order of how they appeared in the superpermutation. Each of those permutation are then placed next to a copy of themselves with an nth symbol added in between the two copies. Finally, each resulting structure is placed next to each other and all adjacent identical symbols are merged."](https://en.wikipedia.org/wiki/Superpermutation#Finding_superpermutations)

There is definitely a lot I could do to optimize the way I wrote this function, particularly in terms of memory, but it is good enough for our purposes. This function is only known to produce the smallest superpermutation for 1 ≤ n ≤ 5, after that the pattern breaks and smaller superpermutations can be found through other means.

In [6]:
sp_six = SuperPermutation(6, intList_to_string(recursive_solver(6)), colorCode)
print(sp_six)

SP: N:6 K:873
123456123451623451263451236451234651234156234152634152364152346152341652341256341253641253461253416253412653412356412354612354162354126354123654123145623145263145236145231645231465231425631425361425316425314625314265314235614235164235146235142635142365142315642315462315426315423615423165423124563124536124531624531264531246531243561243516243512643512463512436512431562431526431524631524361524316524312564312546312543612543162543126543121345621345261345216345213645213465213425613425163425136425134625134265134215634215364215346215342615342165342135642135462135426135421635421365421324561324516324513624513264513246513241563241536241532641532461532416532413562413526413524613524163524136524132564132546132541632541362541326541321456321453621453261453216453214653214356214352614352164352146352143652143256143251643251463251436251432651432156432154632154362154326154321654321


873 is a lot of digits to take in, defintely annoying to analyze by hand. If we are going to explore this subject further and examine a more optimal solution it would be nice to have a function to check if a number is in fact a superpermutation.

In [7]:
import itertools as it

In [33]:
# sequence as int list
def is_sp(sequence):
    n = max(sequence)
    sequence_str = intList_to_string(sequence)
    sub_p = list(it.permutations(range(1, n + 1)))
    sub_p_str = [intList_to_string(x) for x in sub_p]
    for p in sub_p_str:
        if p in sequence_str:
            continue
        else:
            return False
    return True

The above function creates strings from all the permutations 1 to N for the highest value in the input list. It then iterates through these sub permutations and tests if they are in the input sequence. Let's test to see if our recursive solver and out tester are running correctly.

In [35]:
print(is_sp([1, 2, 3, 1, 2, 1]))
print(is_sp(recursive_solver(4)))
print(is_sp(recursive_solver(5)))
print(is_sp(recursive_solver(6)))
print(is_sp(recursive_solver(7)))

False
True
True
True
True


Everything is looking good. For finding smaller supermutations for N > 5 we need a different method than the recursive one. From the video Robin Houston demonstrates a creative method wich translates finding combinations of permutations into a graph search problem. The cleverness of this approach is it takes an unknown problem and translates it into one that is heavily researched, The Travelling Salesman Problem. Any path along the graph, in which each node is a permutation, and which traverses every node, is guaranteed to be a superpermutation. The problem is then is simply to find the shortest path.